# Convert a PyTorch model to Tensorflow using ONNX

In this tutorial, we will show you how to export a model defined in PyTorch to ONNX and then import the ONNX model into Tensorflow to run it. We will also show you how to save this Tensorflow model into a file for later use.  

## Installations

First let's install [ONNX](https://github.com/onnx/onnx), [PyTorch](https://github.com/pytorch/pytorch), and [Tensorflow](https://github.com/tensorflow/tensorflow) by following the instructions on each of their repository.

Then Install torchvision by the following command:
```
pip install torchvision
```

Next install [onnx-tensorflow](https://github.com/onnx/onnx-tensorflow) by the following commands:
```
git clone git@github.com:onnx/onnx-tensorflow.git && cd onnx-tensorflow
pip install -e .
```

## Define model

In this tutorial we are going to use the [MNIST model](https://github.com/pytorch/examples/tree/master/mnist) from PyTorch examples.  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Train and test model
Now let's train this model. By default if GPU is availalbe on your environment, it will use GPU instead of CPU to run the training. In this tutorial we will train this model with 60 epochs. It will takes about 15 minutes on an environment with 1 GPU to complete this training. You can always adjust the number of epoch base on how well you want your model to be trained.

In [ ]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

# Train this model with 60 epochs and after process every 300 batches log the train status 
args = parser.parse_args(['--epochs', '60', '--log-interval', '300'])

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

## Save the trained model

In [ ]:
# Save the trained model to a file
torch.save(model.state_dict(), 'output/mnist.pth')

## Export the trained model to ONNX 
In order to export the model, Pytorch exporter needs to run the model once and save this resulting traced model to a ONNX file. Therefore, we need to provide the input for our MNIST model. Here we would expect to get a black and white 28 x 28 picture as an input to run this model in inference phase.

In [ ]:
from torch.autograd import Variable

# Load the trained model from file
trained_model = Net()
trained_model.load_state_dict(torch.load('output/mnist.pth'))

# Export the trained model to ONNX
dummy_input = Variable(torch.randn(1, 1, 28, 28)) # one black and white 28 x 28 picture will be the input to the model
torch.onnx.export(trained_model, dummy_input, "output/mnist.onnx")

PS. You can examine the graph of this mnist.onnx file using an ONNX viewer call [Netron](https://github.com/lutzroeder/Netron)

## Import the ONNX model to Tensorflow
We will use onnx_tf.backend.prepare to import the ONNX model into Tensorflow.

In [ ]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('output/mnist.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

Let's explore the tf_rep object return from onnx.tf.backend.prepare

In [ ]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

## Run the model in Tensorflow

In [ ]:
import numpy as np
from IPython.display import display
from PIL import Image

print('Image 1:')
img = Image.open('assets/two.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))

print('Image 2:')
img = Image.open('assets/three.png').resize((28, 28)).convert('L')
display(img)
output = tf_rep.run(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :])
print('The digit is classified as ', np.argmax(output))

## Save the Tensorflow model into a file

In [ ]:
tf_rep.export_graph('output/mnist.pb')